In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np


# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

In [2]:
# Loading the shadow lattice from csv
fields_lattice = os.path.relpath('../data/fields/sunlight_access.csv')
field_lattice = tg.lattice_from_csv(fields_lattice)

### stuff

In [3]:
context_path = os.path.relpath('../data/meshes/immediate_context.obj')
context_mesh = tm.load(context_path)
# loading the lattice from csv
lattice = os.path.relpath('../data/meshes/voxelized_envelope_highres.csv')
envelope_lattice = tg.lattice_from_csv(lattice)


In [4]:

# removing values above set treshold and setting them to zero
new_envelope_lattice = field_lattice

# set all usable values to 1

base_lattice = np.where(new_envelope_lattice > 0, 1,new_envelope_lattice)
base_lattice = base_lattice.astype(bool)
base_lattice = tg.to_lattice(base_lattice, field_lattice)
base_lattice.shape

(36, 21, 12)

In [5]:
p = pv.Plotter(notebook=True)
pv.set_plot_theme("dark")

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
# p.add_mesh(grid.outline(), color="grey", label="Domain")

# # adding the availability lattice
# init_avail_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# adding axes
# p.add_axes()
#p.show_bounds(grid="back", location="back", color="#aaaaaa")



<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x0000029C832D90F0) at 0x0000029C8D7386A0>

In [6]:
def create_mesh(value):

    lattice = (field_lattice < value) * envelope_lattice

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([0.9, 1.1])
    # adding the voxels
    p.add_mesh(threshed, color=('white'), name='sphere', show_edges=False, opacity=1.0, show_scalar_bar=False)

    return

### visualizer

In [7]:
p.add_slider_widget(create_mesh, [field_lattice.min(), field_lattice.max()], title='Time', value=0, event_type="always", style="classic")
#plotting
p.show(use_ipyvtk=True)

  File "C:\Users\Cobyb\AppData\Local\Temp/ipykernel_14216/2979341381.py", line 10, in create_mesh
    p.add_mesh(threshed, color=('white'), name='sphere', show_edges=False, opacity=1.0, show_scalar_bar=False)
  File "C:\Users\Cobyb\anaconda3\envs\scds\lib\site-packages\pyvista\plotting\plotting.py", line 1567, in add_mesh
    raise ValueError('Empty meshes cannot be plotted. Input mesh has zero points.')
ValueError: Empty meshes cannot be plotted. Input mesh has zero points.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.6075686833789, 708.1911636833788, 743.2184808333789),
 (65.08283250000001, -12.333572500000002, 22.69374465),
 (0.0, 0.0, 1.0)]